<a href="https://colab.research.google.com/github/mrgrigorii/GUIpygame/blob/master/find_intersections_per_type_v2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import array
from collections import defaultdict
import gzip
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive/data

 chromosomesGRCh37	 'Jupyter — копия.rar'	 sorted_snps_positions_per_type
 hocomoco		  sites
 intersections_per_type   sitesV3


In [ ]:
data_dir = 'drive/My Drive/data'
os.listdir(data_dir)

['chromosomesGRCh37',
 'sites',
 'sitesV3',
 'hocomoco',
 'sorted_snps_positions_per_type',
 'Jupyter — копия.rar',
 'intersections_per_type']

In [ ]:
CHROMOSOMES = [str(i) for i in range(1, 23)] + ['X', 'Y', 'MT']

In [ ]:
def load_hocomoco_motif(file_name):
    motif_list = []
    with open(file_name, 'r') as f:
        data = f.read().split('>')[1:]
        for motif in data:
            motif = motif.split('\n')
            motif_name = motif[0]
            motif_matrix = list(
                list(map(float, i.split())) + [-10.0] for i in motif[1:] if i
            )
            motif_list.append([motif_name, motif_matrix])

    return motif_list

In [ ]:
def load_sites(file_gzip_path):
    with gzip.open(file_gzip_path, 'rb') as f:
        data = f.read().decode("utf-8")
        data = data.strip('\n').split('\n')
        return data

def load_sorted_snps(chromosome, sorted_snp_data_dir):
    #print(chromosome)
    for file_name in os.listdir(sorted_snp_data_dir):
        if file_name.split('_')[-1] == chromosome:
            with open(os.path.join(sorted_snp_data_dir, file_name), 'r') as fin:
                data = list(map(int, fin.read().strip('\n').split('\n')))
            return data
    return None

In [ ]:
def find_intersections_v2(sites, sorted_snps_data):
    intersections = []
    curr_j = 0
    for i in range(len(sites)):
        min_value, max_value, score = sites[i]
        site_pos = min_value
        snps_in_site = 0
        
        positions = {}
        for j in range(curr_j, len(sorted_snps_data)):
            snps_pos = sorted_snps_data[j]
            if snps_pos < min_value:
                curr_j = j
                continue
            elif min_value <= snps_pos <= max_value:
                snps_in_site += 1
                _i = snps_pos - min_value
                if _i not in positions:
                    positions[_i] = 1
                else:
                    positions[_i] += 1
            else:
                break
        if snps_in_site:
            positions_str = []
            for pos, value in positions.items():
                positions_str.append('{}:{}'.format(pos, value))
            positions_str = ','.join(positions_str)
        else:
            positions_str = ''
        intersections.append('{}\t{}\t{}\t{}\n'.format(site_pos, snps_in_site, score, positions_str))
    return intersections

In [ ]:
def save_intersections_data(output_qzip_file, intersections):
    with gzip.open(output_qzip_file, 'wb') as output_file:
        output_file.write(''.join(intersections).encode('utf-8'))

In [ ]:
cancers_sorted_snps_dir = os.path.join(data_dir, 'sorted_snps_positions_per_type')
cancer_types = os.listdir(cancers_sorted_snps_dir)
cancer_types

['breastWGS',
 'headandneckWGS',
 'bloodWGS',
 'esophagusWGS',
 'boneWGS',
 'colorectalWGS',
 'gallbladderWGS',
 'cervixWGS',
 'bladderWGS',
 'brainWGS',
 'nervoussystemWGS',
 'prostateWGS',
 'ovaryWGS',
 'skinWGS',
 'kidneyWGS',
 'mesenchymalWGS',
 'lungWGS',
 'pancreasWGS',
 'liverWGS',
 'stomachWGS',
 'uterusWGS']

In [ ]:
motifs = load_hocomoco_motif(os.path.join(data_dir, 'hocomoco/HOCOMOCOv11_core_pwms_HUMAN_mono.txt'))

In [ ]:
i = start = 0
# GATA6_HUMAN.H11MO.0.A
for motif_name, motif_matrix in motifs[start:]:
    print(i, len(motifs))
    i += 1
    if motif_name not in ['GATA6_HUMAN.H11MO.0.A']: #'AP2A_HUMAN.H11MO.0.A', 'ESR1_HUMAN.H11MO.0.A', 'FOXA1_HUMAN.H11MO.0.A']:
        continue
    motif_sites_data_folder = os.path.join(data_dir, 'sitesV3', motif_name)
    
    motif_len = len(motif_matrix)
    print(motif_name, motif_len)

    chromosomes_sites_paths = os.listdir(motif_sites_data_folder)
    chromosomes_sites_dict = {}
    for chr_file in chromosomes_sites_paths:
        for chromosome in CHROMOSOMES:
            if chr_file.split('_')[0][3:] == chromosome:
                chromosomes_sites_dict[chr_file] = chromosome

    for cancer_type in cancer_types:
        sorted_snp_data_dir = os.path.join(cancers_sorted_snps_dir, cancer_type)
        print(cancer_type)

        intersections_data_folder = os.path.join(
            data_dir,
            'intersections_per_type',
            motif_name, 
            cancer_type,
        )
        if not os.path.exists(intersections_data_folder):
            os.makedirs(intersections_data_folder)


        for chr_file in chromosomes_sites_paths:
            chromosome = chromosomes_sites_dict[chr_file]

            data = load_sites(os.path.join(motif_sites_data_folder, chr_file))
            if len(data) == 1 and data[0] == '':
                print('not data')
                continue
            sites = np.array([list(map(int, x.split('\t'))) for x in data])
            sites = np.stack((sites[:,0], sites[:,0] + motif_len - 1, sites[:,1]), axis=1)

            sorted_snps = load_sorted_snps(chromosome, sorted_snp_data_dir)
            if not sorted_snps:
                continue
            sorted_snps_data = np.array(sorted_snps, dtype=np.int64)

            intersections = find_intersections_v2(sites, sorted_snps_data)
            output_qzip_file = os.path.join(intersections_data_folder, chr_file.split('.')[0] + '_intersections.txt.gz')
            save_intersections_data(output_qzip_file, intersections)

0 402
1 402
2 402
3 402
4 402
5 402
6 402
7 402
8 402
9 402
10 402
11 402
12 402
13 402
14 402
15 402
16 402
17 402
18 402
19 402
20 402
21 402
22 402
23 402
24 402
25 402
26 402
27 402
28 402
29 402
30 402
31 402
32 402
33 402
34 402
35 402
36 402
37 402
38 402
39 402
40 402
41 402
42 402
43 402
44 402
45 402
46 402
47 402
48 402
49 402
50 402
51 402
52 402
53 402
54 402
55 402
56 402
57 402
58 402
59 402
60 402
61 402
62 402
63 402
64 402
65 402
66 402
67 402
68 402
69 402
70 402
71 402
72 402
73 402
74 402
75 402
76 402
77 402
78 402
79 402
80 402
81 402
82 402
83 402
84 402
85 402
86 402
87 402
88 402
89 402
90 402
91 402
92 402
93 402
94 402
95 402
96 402
97 402
98 402
99 402
100 402
101 402
102 402
103 402
104 402
105 402
GATA6_HUMAN.H11MO.0.A 13
breastWGS
headandneckWGS
bloodWGS
esophagusWGS
boneWGS
colorectalWGS
gallbladderWGS
cervixWGS
bladderWGS
brainWGS
nervoussystemWGS
prostateWGS
ovaryWGS
skinWGS
kidneyWGS
mesenchymalWGS
lungWGS
pancreasWGS
liverWGS
stomachWGS
uterusWGS
10

In [ ]:
all(['', 'sdf'])

False